# Using Pushshift Module to extract Submissions Data from Reddit via Python

PRAW is pretty good at gettin reddit data but there are some limitations with it.
Including the removal of the [subreddit.submissions endpoint](https://www.reddit.com/r/changelog/comments/7tus5f/update_to_search_api/.). 

So for extracting Reddit submissions and the primarily data such as upvotes and comments count, I put together this notebook using Pushshift.

If you still prefer PRAW for extract submissions, I have written a code [template here](https://github.com/SeyiAgboola/Seyi_Projects/blob/master/submission_list.py).

I will also host the code on GitHub.

More info on the removal of the [subreddit.submissions endpoint](https://www.reddit.com/r/redditdev/comments/8bia9n/praw_psa_the_subredditsubmissions_method_no/).

# Import modules

In [1]:
import pandas as pd
import requests #Pushshift accesses Reddit via an url so this is needed
import json #JSON manipulation
import csv #To Convert final table into a csv file to save to your machine
import time
import datetime

# Pushshift URL Examples

In [2]:
#We can access the Pushshift API through building an URL with the relevant parameters without even needing Reddit credentials.
#These are some examples. You can follow the links and they will generate a page with JSON data
search_ps4_after_date = "https://api.pushshift.io/reddit/search/submission/?q=screenshot&after=1514764800&before=1517443200&subreddit=PS4"
search_science = "https://api.pushshift.io/reddit/search/submission/?q=science"

# Parameters for your Pushshift URL
These are probably the most important parameters to consider when building your Pushshift URL:

* size — increase limit of returned entries to 1000
* after — where to start the search
* before — where to end the search
* title — to search only within the submission’s title
* subreddit — to narrow it down to a particular subreddit

In [3]:
#Adapted from this https://gist.github.com/dylankilkenny/3dbf6123527260165f8c5c3bc3ee331b
#This function builds an Pushshift URL, accesses the webpage and stores JSON data in a nested list
def getPushshiftData(query, after, before, sub):
    #Build URL
    url_submission = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    url = url_submission
#     url_comment = 'https://api.pushshift.io/reddit/search/comment/?q='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
#     url = url_comment
    #Print URL to show user
    print(url)
    #Request URL
    r = requests.get(url)
    #Load JSON data from webpage into data variable
    data = json.loads(r.text)
    #return the data element which contains all the submissions data
    return data['data']

# Extract key information from Submissions

We want key data for further analysis including: 
* Submission Title
* URL 
* Flair
* Author
* Submission post ID
* Score
* Upload Time
* No. of Comments 
* Permalink.


In [4]:
#This function will be used to extract the key data points from each JSON result
def collectSubData(subm):
    #subData was created at the start to hold all the data which is then added to our global subStats dictionary.
    subData = list() #list to store data points
    #title = subm['title']
    #url = subm['url']
    #flairs are not always present so we wrap in try/except
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    title = subm['title']
    url = subm['url']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
#     body = subm['body']
    #Put all data points into a tuple and append to subData
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair))
#     subData.append((sub_id,author,body,score,created,permalink,flair)) #for comments

    #Create a dictionary entry of current submission data and store all data related to it
    subStats[sub_id] = subData

# Update your Search Settings here

- Create your timestamps and queries for your search URL
- https://www.unixtimestamp.com/index.php > Use this to create your timestamps
- after = "1577836800" #Submissions after this timestamp (1577836800 = 01 Jan 20)
- before = "1607040000" #Submissions before this timestamp (1607040000 = 04 Dec 20)
- query = "Cyberpunk 2077" #Keyword(s) to look for in submissions
- sub = "Games" #Which Subreddit to search in

In [5]:
# after = "1514764800" #01/01/2018 @ 12:00am (UTC)
# before = "1515542400" #01/10/2018 @ 12:00am (UTC)
query = "vaccine" 
sub = "conspiracy"

after = "1577836800" #Submissions after this timestamp (1577836800 = 01 Jan 20)
before = "1607040000" #Submissions before this timestamp (1607040000 = 04 Dec 20)
#before = "1583020800" # 03/01/2020 @ 12:00am (UTC)
# query = "Cyberpunk 2077" #Keyword(s) to look for in submissions
# sub = "Games" #Which Subreddit to search in

#subCount tracks the no. of total submissions we collect
subCount = 0
#subStats is the dictionary where we will store our data.
subStats = {}

In [6]:
# We need to run this function outside the loop first to get the updated after variable
data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date
while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
    after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
    data = getPushshiftData(query, after, before, sub)
    
print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=vaccine&size=1000&after=1577836800&before=1607040000&subreddit=conspiracy
100
2020-02-15 19:28:10
https://api.pushshift.io/reddit/search/submission/?title=vaccine&size=1000&after=1581791290&before=1607040000&subreddit=conspiracy
100
2020-03-16 00:43:06
https://api.pushshift.io/reddit/search/submission/?title=vaccine&size=1000&after=1584315786&before=1607040000&subreddit=conspiracy
100
2020-03-29 16:22:07
https://api.pushshift.io/reddit/search/submission/?title=vaccine&size=1000&after=1585491727&before=1607040000&subreddit=conspiracy
100
2020-04-07 03:43:21
https://api.pushshift.io/reddit/search/submission/?title=vaccine&size=1000&after=1586223801&before=1607040000&subreddit=conspiracy
100
2020-04-12 23:28:42
https://api.pushshift.io/reddit/search/submission/?title=vaccine&size=1000&after=1586726922&before=1607040000&subreddit=conspiracy
100
2020-04-19 00:48:20
https://api.pushshift.io/reddit/search/submission/?title=vaccine&size=

# Check your Submission Extraction was successful

In [7]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

2905 submissions have added to list
1st entry is:
Vaccine Injury Attorney Destroys Connecticut Lawmakers created: 2020-01-01 21:07:32
Last entry is:
Any thoughts on this? I understand (besides Trump) they were the last three presidents but they always seem to have a really good connection. I personally don’t think much of the vaccine in a negative way, but curious for any thoughts ... created: 2020-12-03 16:01:44


# Save data to CSV file

In [8]:
def updateSubs_file():
    upload_count = 0
    #location = "\\Reddit Data\\" >> If you're running this outside of a notebook you'll need this to direct to a specific location
    #print("input filename of submission file, please add .csv")
    #filename = input() #This asks the user what to name the file
#     filename = "comments_vaccine.csv"
    filename = "submissions_vaccine.csv"
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
#         headers = ["Post ID","Author","Body","Score","Publish Date","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
updateSubs_file()

2905 submissions have been uploaded
